--------------
Get Data:

In [52]:
g = lambda x: [y.strip() for y in x]

In [53]:
classRooms = [
    "A1.04 	41",
    "A1.06 	22",
    "A1.08 	20",
    "A1.10 	56",
    "B0.201 	48",
    "C0.110 	117",
    "C1.112 	60",]

classRooms = [g(x.split("\t")) for x in classRooms]

In [54]:
subjects = """Advanced Heuristics 	1 	0 	nvt 	1 	10
Algoritmen en complexiteit 	1 	1 	25 	1 	25
Analysemethoden en -technieken 	1 	0 	nvt 	0 	nvt
Architectuur en computerorganisatie 	2 	0 	nvt 	0 	nvt
Autonomous Agents 2 	2 	1 	10 	1 	10
Bioinformatica 	3 	1 	20 	1 	20
Calculus 2 	1 	1 	40 	0 	nvt
Collectieve Intelligentie 	3 	1 	20 	1 	20
Compilerbouw 	2 	1 	40 	1 	40
Compilerbouw (practicum) 	0 	0 	nvt 	1 	15
Data Mining 	2 	1 	10 	1 	10
Databases 2 	1 	1 	40 	0 	nvt
Heuristieken 1 	1 	1 	25 	0 	nvt
Heuristieken 2 	1 	1 	20 	0 	nvt
Informatie- en organisatieontwerp 	2 	1 	15 	1 	15
Interactie-ontwerp 	2 	0 	nvt 	0 	nvt
Kansrekenen 2 	2 	0 	nvt 	0 	nvt
Lineaire Algebra 	2 	0 	nvt 	0 	nvt
Machine Learning 	2 	0 	nvt 	0 	nvt
Moderne Databases 	1 	1 	20 	1 	20
Netwerken en systeembeveiliging 	0 	0 	nvt 	1 	20
Programmeren in Java 2 	0 	0 	nvt 	1 	20
Project Genetic Algorithms 	0 	0 	nvt 	1 	15
Project Numerical Recipes 	0 	0 	nvt 	1 	15
Reflectie op de digitale cultuur 	2 	1 	20 	0 	nvt
Software engineering 	1 	1 	40 	1 	40
Technology for games 	2 	1 	20 	0 	nvt
Webprogrammeren en databases 	2 	1 	20 	1 	20
Zoeken, sturen en bewegen 	0 	0 	nvt 	1 	15"""

subjects = [g(x.split("\t")) for x in subjects.split("\n")]

In [55]:
students = []

import csv

with open('studenten_roostering.csv', 'r') as csvfile:
    r = csv.reader(csvfile, delimiter=',', quotechar='|')
    
    # Skip title
    next(r, None)
    
    for row in r:
        students.append(row)

--------------
Classes:

In [56]:
class ClassRoom:
    def __init__(self, room_number, capacity):
        self.room_number = room_number
        self.capacity = capacity
    
    def __str__(self):
        return self.room_number

In [68]:
class Subject:
    def __init__(self, name, n_lectures, n_workLectures, w_maxStud, 
                 n_praticas, p_maxStud):
        
        self.name = name
        
        self.lectures = []
        self.workLectures = []
        self.praticas = []
        
        for i in range(int(n_lectures)):
            self.lectures.append(Lecture(i, "nvt"))
            
        for i in range(int(n_workLectures)):
            self.workLectures.append(Lecture(i, w_maxStud))
            
        for i in range(int(n_praticas)):
            self.praticas.append(Lecture(i, p_maxStud))
        
        self.students = []
        
    def __str__(self):
        return self.name

In [58]:
class Lecture:
    def __init__(self, lecture_number, maxStud):
        self.lecture_number = lecture_number
        
        if maxStud == "nvt":
            self.maxStud = 0
        else:
            self.maxStud = maxStud
            
        self.maxStud = maxStud
        
    def __str__(self):
        return "%s %s %s" % (type(self), self.lecture_number, self.maxStud)
    

In [59]:
class Student:
    def __init__(self, surname, name, studentId, subject1, subject2, 
                 subject3, subject4, subject5, subject_dct):
        
        self.surname = surname
        self.name = name
        self.studentId = studentId
        
        self.subjectNames = []
        self.__addSubject(subject1)
        self.__addSubject(subject2)
        self.__addSubject(subject3)
        self.__addSubject(subject4)
        self.__addSubject(subject5)
        
        self.subjects = []
        
        self.__clean_up()
        self.__fillInSubject(subject_dct)
        self.__addStudentToSubject()
        
    def __str__(self):
        return "%s %s %s" % (self.name, self.surname, self.studentId)
    
    def __addSubject(self, subject):
        if subject != "":
            self.subjectNames.append(subject)
    
    def __addStudentToSubject(self):
        for subject in self.subjects:
            subject.students.append(self)
    
    def __clean_up(self):
        # Fixes CSV errors
        
        for i, x in enumerate(self.subjectNames):
            if x == "Zoeken":
                self.subjectNames[i] = "Zoeken, sturen en bewegen"
            if x == " sturen en bewegen":
                del self.subjectNames[i]
            if x == "Compilerbouw practicum":
                self.subjectNames[i] = "Compilerbouw (practicum)"
            if x == "Informatie- en organsatieontwerp":
                self.subjectNames[i] = "Informatie- en organisatieontwerp"
                
    def __fillInSubject(self, subject_dct):
        for subjectName in self.subjectNames:
            self.subjects.append(subject_dct[subjectName])

--------------
Transform data:

In [60]:
classRooms = [ClassRoom(x[0], x[1]) for x in classRooms]

subjects = [Subject(x[0], x[1], x[2], x[3], x[4], x[5]) for x in subjects]
subject_dct = {x.__str__(): x for x in subjects}

students = [Student(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], subject_dct) 
            for x in students]


In [67]:
i=3

print(subjects[i])
[x.__str__() for x in subjects[i].workLectures]

Architectuur en computerorganisatie


[]

In [62]:
# Laat alle studenten in Algoritmen en complexiteit zien:

print(subjects[1])
[x.__str__() for x in subjects[1].students]

Algoritmen en complexiteit


['Edin Bastings 82689818',
 'Mattijs Baudoin 81385263',
 'Angèle Berentsen 78136904',
 'Emil Bergmans 8522855',
 'Saba Bhaggoe 41585485',
 'Narin Boshuis 19540225',
 'Ethan Broer 14671406',
 'Wytske Buining 86988337',
 'Lucienne Caris 12077419',
 'Manuël de Boon 46701583',
 'Janniek de Leest 66433883',
 'Brigitta Diallo 50143947',
 'André Dieters 57467488',
 'Ko Droste 26260141',
 'Jurrian Fekkes 27298319',
 'Yalda Grootenboer 72895450',
 'Frédéric IJkema 77376714',
 'Jenske Jongejan 15902055',
 'Chi Liebregts 88596522',
 'Fredrika Lopez 21095566',
 'Lisse Moes 53585410',
 'Inas Niezen 67817269',
 'Vincentius Pater 34765676',
 'Solane Paulissen 1665980',
 'Rémy Piek 5976174',
 'Timo Postuma 36903574',
 'Desireé Potgieter 64947290',
 'Cheyenna Reuvekamp 3474870',
 'Karam Rinkel 41527759',
 'Ziya Rooijmans 43902015',
 'Tevfik Rothuizen 40343779',
 'Logan Sonnemans 91527663',
 'Franke Stoppels 64554442',
 'Ard Tielemans 21964491',
 'Deejay van der Geer 51566595',
 'Ay?egül van der Marel 5